Adapted from https://bas.codes/posts/python-sudoku-solver-z3

In [23]:
from pprint import pprint

In [24]:
from z3 import *

"""
Apples are sold in packs of 4.
The price of a pack is $20.
If we have $40 in our pocket, how many packs can we buy?
"""

price = Int("price")
quantity = Int("quantity")

s = Solver()
s.add(4 * price == 20)
s.add(price * quantity == 40)
print(s.check())
print(s.model())

sat
[price = 5, quantity = 8]


In [25]:
# Create an integer variable for each cell in the 9x9 sudoku grid
X = [ [ Int(f"x_%s_%s" % (i+1, j+1)) for j in range(9) ] 
      for i in range(9) ]
pprint(X)

[[x_1_1, x_1_2, x_1_3, x_1_4, x_1_5, x_1_6, x_1_7, x_1_8, x_1_9],
 [x_2_1, x_2_2, x_2_3, x_2_4, x_2_5, x_2_6, x_2_7, x_2_8, x_2_9],
 [x_3_1, x_3_2, x_3_3, x_3_4, x_3_5, x_3_6, x_3_7, x_3_8, x_3_9],
 [x_4_1, x_4_2, x_4_3, x_4_4, x_4_5, x_4_6, x_4_7, x_4_8, x_4_9],
 [x_5_1, x_5_2, x_5_3, x_5_4, x_5_5, x_5_6, x_5_7, x_5_8, x_5_9],
 [x_6_1, x_6_2, x_6_3, x_6_4, x_6_5, x_6_6, x_6_7, x_6_8, x_6_9],
 [x_7_1, x_7_2, x_7_3, x_7_4, x_7_5, x_7_6, x_7_7, x_7_8, x_7_9],
 [x_8_1, x_8_2, x_8_3, x_8_4, x_8_5, x_8_6, x_8_7, x_8_8, x_8_9],
 [x_9_1, x_9_2, x_9_3, x_9_4, x_9_5, x_9_6, x_9_7, x_9_8, x_9_9]]


In [26]:
# Each cell must contain a digit (1 to 9)
cells_c  = [ And(1 <= X[i][j], X[i][j] <= 9) 
             for i in range(9) for j in range(9) ]
pprint(cells_c)

[And(x_1_1 >= 1, x_1_1 <= 9),
 And(x_1_2 >= 1, x_1_2 <= 9),
 And(x_1_3 >= 1, x_1_3 <= 9),
 And(x_1_4 >= 1, x_1_4 <= 9),
 And(x_1_5 >= 1, x_1_5 <= 9),
 And(x_1_6 >= 1, x_1_6 <= 9),
 And(x_1_7 >= 1, x_1_7 <= 9),
 And(x_1_8 >= 1, x_1_8 <= 9),
 And(x_1_9 >= 1, x_1_9 <= 9),
 And(x_2_1 >= 1, x_2_1 <= 9),
 And(x_2_2 >= 1, x_2_2 <= 9),
 And(x_2_3 >= 1, x_2_3 <= 9),
 And(x_2_4 >= 1, x_2_4 <= 9),
 And(x_2_5 >= 1, x_2_5 <= 9),
 And(x_2_6 >= 1, x_2_6 <= 9),
 And(x_2_7 >= 1, x_2_7 <= 9),
 And(x_2_8 >= 1, x_2_8 <= 9),
 And(x_2_9 >= 1, x_2_9 <= 9),
 And(x_3_1 >= 1, x_3_1 <= 9),
 And(x_3_2 >= 1, x_3_2 <= 9),
 And(x_3_3 >= 1, x_3_3 <= 9),
 And(x_3_4 >= 1, x_3_4 <= 9),
 And(x_3_5 >= 1, x_3_5 <= 9),
 And(x_3_6 >= 1, x_3_6 <= 9),
 And(x_3_7 >= 1, x_3_7 <= 9),
 And(x_3_8 >= 1, x_3_8 <= 9),
 And(x_3_9 >= 1, x_3_9 <= 9),
 And(x_4_1 >= 1, x_4_1 <= 9),
 And(x_4_2 >= 1, x_4_2 <= 9),
 And(x_4_3 >= 1, x_4_3 <= 9),
 And(x_4_4 >= 1, x_4_4 <= 9),
 And(x_4_5 >= 1, x_4_5 <= 9),
 And(x_4_6 >= 1, x_4_6 <= 9),
 And(x_4_7

In [27]:
# Every digit has to be placed exactly once in each row
rows_c   = [ Distinct(X[i]) for i in range(9) ]

# Every digit has to be placed exactly once in each column
cols_c   = [ Distinct([ X[i][j] for i in range(9) ]) 
             for j in range(9) ]

# Every digit has to be placed exactly once in each 3x3 subgrid
sq_c = [ Distinct([ X[3*i0 + i][3*j0 + j] 
                    for i in range(3) for j in range(3) ]) 
         for i0 in range(3) for j0 in range(3) ]

In [28]:
# Chain all the conditions together
sudoku_c = cells_c + rows_c + cols_c + sq_c

In [29]:
# Load our inital values
instance = ((5,3,0,0,7,0,0,0,0),
            (6,0,0,1,9,5,0,0,0),
            (0,9,8,0,0,0,0,6,0),
            (8,0,0,0,6,0,0,0,3),
            (4,0,0,8,0,3,0,0,1),
            (7,0,0,0,2,0,0,0,6),
            (0,6,0,0,0,0,2,8,0),
            (0,0,0,4,1,9,0,0,5),
            (0,0,0,0,8,0,0,7,9))

instance_c = [ If(instance[i][j] == 0, 
                  True, 
                  X[i][j] == instance[i][j]) 
               for i in range(9) for j in range(9) ]

In [30]:
# Create a solver object and ask z3 to solve the problem
s = Solver()                                            # (1)
s.add(sudoku_c + instance_c)                            # (2)
if s.check() == sat:                                    # (3)
    m = s.model()                                       # (4)
    r = [ [ m.evaluate(X[i][j]) for j in range(9) ]     # (5)
          for i in range(9) ]
    print_matrix(r)                                     # (6)
else:
    print("failed to solve")                            # (7)

[[5, 3, 4, 6, 7, 8, 9, 1, 2],
 [6, 7, 2, 1, 9, 5, 3, 4, 8],
 [1, 9, 8, 3, 4, 2, 5, 6, 7],
 [8, 5, 9, 7, 6, 1, 4, 2, 3],
 [4, 2, 6, 8, 5, 3, 7, 9, 1],
 [7, 1, 3, 9, 2, 4, 8, 5, 6],
 [9, 6, 1, 5, 3, 7, 2, 8, 4],
 [2, 8, 7, 4, 1, 9, 6, 3, 5],
 [3, 4, 5, 2, 8, 6, 1, 7, 9]]
